<a href="https://colab.research.google.com/github/wesley34/comp3414_course_material/blob/master/assignment3_template/Dueling_template_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
# N-step , Dueling DQN
!pip install tensorboardX

     |████████████████████████████████| 317kB 4.6MB/s 


In [2]:
!pip install ptan

     |████████████████████████████████| 773.1MB 23kB/s 
  Created wheel for ptan: filename=ptan-0.6-cp36-none-any.whl size=23502 sha256=cbbd6bac3a685791eea34b36d3dee94fed898bc6fa31ebc9c606bab4111d9518
  Stored in directory: /root/.cache/pip/wheels/f0/4b/2f/9a45fd39b0a614a2716bc6128a7f1adb4647f323a2d90783f2
Successfully built ptan
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101


In [164]:
import torch
import torch.nn as nn
import torch.optim as optim
import ptan
from tensorboardX import SummaryWriter

import numpy as np
import gym

import sys
import math
import collections
import time

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## N step DQN

In [119]:
# warning : when you use n>1 , your model cannot converge to the optimal 
# but faster to converge to some point near the optimal
n = 2
print("You used {}-step DQN framework".format(n))

You used 2-step DQN framework


## Dueling DQN
We create Q-value by Q(s,a) = V(s) + A(s,a) - mean(A(s,k))



In [133]:
class DuelingDQNModel(nn.Module):
  def __init__(self,input_shape,n_actions):
    super(DuelingDQNModel,self).__init__()


    self.conv_net = nn.Sequential(
        nn.Conv2d(input_shape[0],40,kernel_size=8,stride=3),
        nn.ReLU(),
        nn.Conv2d(40,64,kernel_size=4,stride=2),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=2,stride=1),
        nn.ReLU(),
        # nn.BatchNorm2d(),
     
    )

    self.flattened_size = self._get_conv_output_shape(input_shape)

    self.fully_connected_advantage_net = nn.Sequential(
        nn.Linear(self.flattened_size,512),
        nn.ReLU(),
        nn.Linear(512,n_actions)
    )

    self.fully_connected_value_net = nn.Sequential(
        nn.Linear(self.flattened_size,512),
        nn.ReLU(),
        nn.Linear(512,1),
    )

 

  def _get_conv_output_shape(self, shape):
    dummy = torch.zeros(1, *shape)
    output = self.conv_net(dummy)
    return int(np.prod(output.size()))

  #overriden
  def forward(self,x):
    normalized_x = x.float()/256
    conv_out = self.conv_net(normalized_x).view(normalized_x.size()[0],-1) # flatten
    value = self.fully_connected_value_net(conv_out)
    advantage = self.fully_connected_advantage_net(conv_out)
    return value+(advantage - advantage.mean(dim=1,keepdim=True)) # the equation

device setup, change run time in colab from None to GPU 

In [135]:
device = torch.device("cuda") #cuda mean using gpu

create gym

In [136]:
GYM_NAME = "PongNoFrameskip-v4" 
env = gym.make(GYM_NAME)
print("Action avaliable: ")
env.unwrapped.get_action_meanings()

Action avaliable: 


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [137]:
# we will use others' wrapper 
# similar as last time
env = ptan.common.wrappers.wrap_dqn(env)

create network and target network

In [159]:
## tune this please
params = {'replay_size':      100000, # not recommend
        'replay_initial':   10000, # not recommend
        'target_net_sync':  1000, # at least larger
        'epsilon_frames':   10**5, # not recommend
        'epsilon_start':    1.0, # not recommend
        'epsilon_final':    0.02, # can be slightly larger/smaller
        'learning_rate':    0.0001, # smaller the better
        'gamma':   0.99, #
        'batch_size':32,
        "stop_reward": 15} # the game will end if you get stop_reard
        

In [139]:
net = DuelingDQNModel(env.observation_space.shape,env.action_space.n).to(device)
# # reload
# try:
#   net.load_state_dict(torch.load(GYM_NAME+"-best.dat"))
# except:
#   print("You do not have any model trained before")

In [140]:

# 1. loss function agent-target_network 
# 2. target_network sync with agent weighings 

# target_network 
target_network = ptan.agent.TargetNet(net)
# epsilon greedy selector
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params['epsilon_start'])
# agent_network
agent = ptan.agent.DQNAgent(net, selector, device=device)


In [141]:
# check library, it will output the steps-sequence , those sequence is generated by our env and our agent
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=params['gamma'], steps_count=n)
buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=params['replay_size'])
optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])


## Loss function

In [142]:
def unpack_batch(batch):
  # sort the batch and return states, actions, rewards, dones, last_states 
  states, actions, rewards, is_dones, last_states = [], [], [], [], []
  for experience in batch:
    state = np.array(experience.state,copy=False) # deep copy is needed
    # sort the array here
    states.append(state)
    actions.append(experience.action)
    rewards.append(experience.reward)
    is_dones.append(experience.last_state is None)

    if experience.last_state is None:
      last_states.append(state) # append dummy state, as we will set them to 0
    else:
      last_states.append(np.array(experience.last_state, copy=False)) # append experience last_states sources
  return np.array(states, copy=False), np.array(actions), np.array(rewards, dtype=np.float32), \
           np.array(is_dones, dtype=np.uint8), np.array(last_states, copy=False)

In [143]:
def get_loss(batch,net,target_network,GAMMA=params['gamma'],device="cpu"):
  states,actions,rewards,is_dones,next_states = unpack_batch(batch)
  states_v = torch.tensor(states).to(device)
  actions_v = torch.tensor(actions).to(device)
  rewards_v = torch.tensor(rewards).to(device)
  is_dones_v = torch.ByteTensor(is_dones).to(device)
  next_states_v = torch.tensor(next_states).to(device)

 
  
  # current network 

  batch_action_table_output = net(states_v) # 2D array [batch_size,action_size] batch size * action size
  batch_best_action = batch_action_table_output.gather(1,actions_v.unsqueeze(-1)) # if action take is [1,2,0] for batch [1,2,3] ,then we get the value of
                                  # batch_action_table_output[1][1],batch_action_table_output[2][2],batch_action_table_output[3][0]
                                  # check out the function of unsqueeze yourself
  batch_best_action_expanded = batch_best_action.squeeze(-1) # we need to add 1 more dimension on the array 
  current_state_action_value = batch_best_action_expanded
  

  # target network
  # same theory as above but maximize is enough
  next_state_batch_action_table_output = target_network(next_states_v)
  next_state_values = next_state_batch_action_table_output.max(1)[0] # [0] is used as next_state_batch_action_table_output = [[1,2,3,4]]

  # next_state_values = 0 if it is end already, we will not get any future value after it is die
  next_state_values[is_dones_v] = 0.0

  # we do not train target_net work so we need to detach it from the graph 
  next_state_values = next_state_values.detach()

  # next_best_score * Gamma + current_transitional_reward
  expected_state_action_value = next_state_values * GAMMA + rewards_v

  loss_value = nn.MSELoss()(current_state_action_value,expected_state_action_value)
  return loss_value

## Epsilon tracker

In [144]:
# it helps decay the epsilon
class EpsilonTracker:
  def __init__(self, epsilon_greedy_selector, params):
      self.epsilon_greedy_selector = epsilon_greedy_selector
      self.epsilon_start = params['epsilon_start']
      self.epsilon_final = params['epsilon_final']
      self.epsilon_frames = params['epsilon_frames']
      self.frame(0)

  def frame(self, frame):
      self.epsilon_greedy_selector.epsilon = max(self.epsilon_final, self.epsilon_start - frame / self.epsilon_frames)

In [145]:
epsilon_tracker = EpsilonTracker(selector,params)

## Reward Tracker

In [151]:
class RewardTracker:
    def __init__(self, writer, stop_reward):
        self.writer = writer
        self.stop_reward = stop_reward

    def __enter__(self):
        self.ts = time.time()
        self.ts_frame = 0
        self.total_rewards = []
        return self

    def __exit__(self, *args):
        self.writer.close()

    def reward(self, reward, frame, epsilon=None):
        self.total_rewards.append(reward)
        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()
        mean_reward = np.mean(self.total_rewards[-100:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        print("%d: done %d games, mean reward %.3f, speed %.2f f/s%s" % (
            frame, len(self.total_rewards), mean_reward, speed, epsilon_str
        ))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("speed", speed, frame)
        self.writer.add_scalar("reward_100", mean_reward, frame)
        self.writer.add_scalar("reward", reward, frame)
        if mean_reward > self.stop_reward:
            print("Solved in %d frames!" % frame)
            return True
        return False
# define writer
writer = SummaryWriter(comment="-" + "Pong-dueling")

## Training epoch
### Capture the final mean reward (mean 100)

In [173]:
frame_idx = 0

with RewardTracker(writer, params['stop_reward']) as reward_tracker:
    while True:
        frame_idx += 1
        buffer.populate(1)
        epsilon_tracker.frame(frame_idx)

        new_rewards = exp_source.pop_total_rewards()
        if new_rewards:
          if reward_tracker.reward(new_rewards[0], frame_idx, selector.epsilon):
            break

        if len(buffer) < params['replay_initial']:
          continue

        optimizer.zero_grad()
        batch = buffer.sample(params['batch_size'])
        loss_v = get_loss(batch, net, target_network.target_model, GAMMA=params['gamma'], device=device)
        loss_v.backward()
        optimizer.step()
      

        if frame_idx % params['target_net_sync'] == 0:
          target_network.sync()

954: done 1 games, mean reward -20.000, speed 80.28 f/s, eps 0.99
1967: done 2 games, mean reward -19.500, speed 80.34 f/s, eps 0.98
2901: done 3 games, mean reward -20.000, speed 80.88 f/s, eps 0.97
3994: done 4 games, mean reward -19.750, speed 81.10 f/s, eps 0.96
4806: done 5 games, mean reward -20.000, speed 81.50 f/s, eps 0.95
5779: done 6 games, mean reward -19.833, speed 81.22 f/s, eps 0.94
6737: done 7 games, mean reward -20.000, speed 81.13 f/s, eps 0.93
7635: done 8 games, mean reward -20.125, speed 81.06 f/s, eps 0.92
8454: done 9 games, mean reward -20.222, speed 80.97 f/s, eps 0.92
9677: done 10 games, mean reward -19.900, speed 81.01 f/s, eps 0.90
10637: done 11 games, mean reward -20.000, speed 81.10 f/s, eps 0.89
11725: done 12 games, mean reward -20.000, speed 80.69 f/s, eps 0.88
12791: done 13 games, mean reward -20.000, speed 81.37 f/s, eps 0.87
13843: done 14 games, mean reward -20.071, speed 79.86 f/s, eps 0.86
15021: done 15 games, mean reward -19.933, speed 80.88

In [ ]:
print(net)